In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import os

os.chdir("..")

/home/mkcmtcc-user/openr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:


os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

compute_dtype = torch.bfloat16
device   = 'cuda'
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

# model     = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=compute_dtype, attn_implementation="sdpa", device_map=device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
dataset = load_dataset("dmitriihook/numina-deepseek-r1-qwen-7b")["train"]

In [5]:
tokenizer.chat_template = tokenizer.chat_template.replace("{% if '</think>' in content %}{% set content = content.split('</think>')[-1] %}{% endif %}", "")

In [6]:
import re

data = []

c = 0
t = 0

for i, row in enumerate(dataset):
    messages = row["distilabel_metadata"]["raw_input_text_generation_0"]
    generation = row["generation"]

    messages = [
        messages[0],
        {"content": generation, "role": "assistant"}
    ]

    pattern = r"\\boxed\{(.*?)\}"

    solution = row["solution"]
    correct_answer = re.findall(pattern, solution)
    model_answer = re.findall(pattern, generation)

    correct = False

    if len(model_answer) > 0 and len(correct_answer) > 0:
        if model_answer[0] == correct_answer[0]:
            correct = True
            c += 1

    t += 1

    chat    = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=False, return_tensors="pt")

    think_pos = torch.where(chat[0] == 151649)[0]
    if think_pos.numel() > 0:
        data.append({
            "result": "success",
            "pos": think_pos.item(),
            "len": len(chat[0]),
            "id": i,
            "correct": correct
        })
    else:
        data.append({
            "result": "fail",
            "pos": None,
            "len": None,
            "id": i,
            "correct": correct
        })

import json

print(c / t)

with open("positions.json", "w") as f:
    json.dump(data, f)

0.6161616161616161


In [11]:
dataset

Dataset({
    features: ['problem', 'solution', 'messages', 'generation', 'distilabel_metadata', 'model_name'],
    num_rows: 396
})